## Detection on Live Webcam

In [1]:
#import required packages
import tensorflow_hub as hub
import cv2
import numpy
import tensorflow as tf
import pandas as pd

In [2]:
#load pre-trained model and labels
print("[INFO] Loading Mode.....")
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")
print("[INFO] Finished Model Loading....")

[INFO] Loading Mode.....
[INFO] Finished Model Loading....


In [3]:
#load csv file which contains labels of the multiple classes
labels = pd.read_csv('labels.csv', sep=';', index_col="ID")
labels = labels['OBJECT (2017 REL.)']

#loading camera object
cap = cv2.VideoCapture(0)

FileNotFoundError: [Errno 2] No such file or directory: 'labels.csv'

In [ ]:
#set dimension of image to resize
width = 512
height = 512

while(True):
    #capute frame by frame
    ret, frame = cap.read()
    
    #resize to respect the input-shape
    inp = cv2.resize(frame, (width, height))
    
    #convert img to RGB (as OpenCV read image as BGR format)
    rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)
    
    #convert img to tensor image
    rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)
    
    #add dims to rgb_tensor
    rgb_tensor = tf.expand_dims(rgb_tensor, 0)
    
    #the output dictionaries of the detector
    boxes, scores, classes, num_detections = detector(rgb_tensor)
    
    #get the predicted labels
    pred_labels = classes.numpy().astype('int')[0]
    pred_labels = [labels[i] for i in pred_labels]
    
    #get the predicted bounding boxes location
    pred_boxes = boxes.numpy()[0].astype('int')
    
    #get the predicted scores
    pred_scores = scores.numpy()[0]
    
    #copy the input image to draw bounding boxes
    result = inp.copy()
    
    #loop throughout the detections and place a box around it
    for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        
        #set the minimum confidence as 0.5
        if score < 0.5:
            continue
            
        text = "{}: {:.2f}%".format(label, score*100)
        cv2.rectangle(result,(xmin,ymax),(xmax,ymin),(0,255,0),2)
        y = ymin - 20 if ymin - 20 > 20 else ymin + 20
        cv2.putText(result, text, (xmin,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
        
    #display the resulting frame
    cv2.imshow('Result frame', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
#when every thing done, release the caputer
cap.release()
    